In [1]:
import igraph
import matplotlib.pyplot as plt2

ModuleNotFoundError: No module named 'igraph'

In [2]:
labels = ['Rook', 'Knight', 'Bishop', 'Pawn', 'King', 'Queen']
adjacency_matrix = [
    [0, 0, 0, 5, 0, 0],
    [2, 0, 0, 3, 0, 0],
    [0, 1, 0, 3, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 4, 7, 0, 1],
    [0, 0, 4, 7, 0, 0]
]
edges = []
weights = []
for i in range(len(labels)):
    for j in range(len(labels)):
        if adjacency_matrix[i][j] > 0:
            edges.append((i, j))
            weights.append(adjacency_matrix[i][j])
print(edges)
print(weights)

[(0, 3), (1, 0), (1, 3), (2, 1), (2, 3), (4, 2), (4, 3), (4, 5), (5, 2), (5, 3)]
[5, 2, 3, 1, 3, 4, 7, 1, 4, 7]


In [ ]:
g = igraph.Graph(directed=True)

g.add_vertices(6)
g.vs['label'] = labels

g.add_edges(edges)
g.es['weight'] = weights

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
igraph.plot(
    g,
    bbox = (300,300),
    target=ax,
    # layout="graphopt",  # kamada_kawai
    vertex_label_color = 'black',
    vertex_label_size = 10,
    vertex_size = 55,
    vertex_color = 'white',
    edge_arrow_width=7,
    edge_width=1,
    edge_label=g.es["weight"],
    # edge_align_label=True,
    # edge_background='white'
)
plt.show()

## Массив записей

In [34]:
class ArrayRecordsGraph:
    def __init__(self, adjacency_matrix: list[list[int]], labels: list[str]):
        self.adjacency_matrix = adjacency_matrix
        self.labels = labels
        self.records = []
        self._set_records_from_adjacency_matrix()

    def _set_records_from_adjacency_matrix(self) -> None:
        for index in range(len(self.labels)):
            kids = self._get_kids_from_adjacency_matrix(index)
            parents = self._get_parents_from_adjacency_matrix(index)
            neighbours = sorted(kids + parents)
            self.records.append({
                "index": index,
                "name": labels[index],
                "count_kids": len(kids),
                "count_parents": len(parents),
                "count_neighbours": len(neighbours),
                "kids": kids,
                "parents": parents,
                "neighbours": neighbours,
            })

    def _get_kids_from_adjacency_matrix(self, parent_index: int) -> list[tuple]:
        kids = []
        for i in range(len(adjacency_matrix)):
            if self.adjacency_matrix[parent_index][i] > 0:
                kids.append((i, self.labels[i], self.adjacency_matrix[parent_index][i]))
        return kids

    def _get_parents_from_adjacency_matrix(self, kid_index: int) -> list[tuple]:
        parents = []
        for i in range(len(adjacency_matrix)):
            if self.adjacency_matrix[i][kid_index] > 0:
                parents.append((i, self.labels[i], self.adjacency_matrix[i][kid_index]))
        return parents

    def get_neighbours_by_index(self, index: int) -> list[int]:
        assert 0 <= index < len(self.labels)
        return [neighbour[0] for neighbour in self.records[index]['neighbours']]
    
    def get_kids_by_index(self, index: int) -> list[int]:
        assert 0 <= index < len(self.labels)
        return [kid[0] for kid in self.records[index]['kids']]

    def get_parents_by_index(self, index: int) -> list[int]:
        assert 0 <= index < len(self.labels)
        return [kid[0] for kid in self.records[index]['parents']]

    def is_path_by_names(self, path: list[str]) -> bool:
        path = [self.get_index_by_name(vertex) for vertex in path]
        return self.is_path_by_indexes(path)

    def is_path_by_indexes(self, path: list[int]) -> bool:
        last_kids = [path[0]]  # start position
        last_vertex = path[0]
        edges = set()
        for index in path:
            record = self.records[index]
            if index not in last_kids:
                return False
            edge = (last_vertex, index)
            if edge in edges:
                return False
            edges.add(edge)
            last_vertex = index
            last_kids = self.get_kids_by_index(index)
        return True

    def is_full_subgraph_by_names(self, vertexes: list[str]) -> bool:
        vertexes = [self.get_index_by_name(vertex) for vertex in vertexes]
        return self.is_full_subgraph(vertexes)
    
    def is_full_subgraph(self, vertexes: list[int])-> bool:
        for vertex in vertexes:
            neighbours = self.get_neighbours_by_index(vertex)
            for check_vertex in vertexes:
                if check_vertex == vertex:
                    continue
                if check_vertex not in neighbours:
                    return False
        return True

    def get_name_by_index(self, index: int) -> str:
        assert 0 <= index < len(self.labels)
        return self.labels[index]

    def get_index_by_name(self, name: str) -> int:
        return self.get_record_by_name(name)['index']

    def get_record_by_name(self, name: str) -> dict:
        for record in self.records:
            if record['name'] == name:
                return record
        raise ValueError

    def get_indexes_sum_incident_edges_more_than(self, criterion: int) -> list[int]:
        filtered = []
        for record in self.records:
            sum_incident_edges = sum(neighbour[2] for neighbour in record['neighbours'])
            if sum_incident_edges > criterion:
                filtered.append(record['index'])
        return filtered

    def get_count_of_edges(self) -> int:
        count_of_neighbours = sum(record['count_neighbours'] for record in self.records)
        return count_of_neighbours // 2



In [35]:
graph = ArrayRecordsGraph(adjacency_matrix, labels)

print(graph.get_kids_by_index(1))          # [0, 3]
print(graph.get_parents_by_index(1))       # [2]
print(graph.get_neighbours_by_index(1))    # [0, 2, 3]

print("Count of edges:", graph.get_count_of_edges())  # 10

print(graph.get_index_by_name("Knight"))   # 1

print(graph.get_indexes_sum_incident_edges_more_than(0))  # [0, 5]
print(graph.get_indexes_sum_incident_edges_more_than(12))  # 3
print(graph.get_indexes_sum_incident_edges_more_than(7))  # 2, 3, 4, 5
print(graph.get_indexes_sum_incident_edges_more_than(6))  # 0, 2, 3, 4, 5

is_path_tests = [
    (["King", "Queen", "Bishop", "Knight", "Rook", "Pawn"], True),
    (["King", "Pawn"], True),
    (["King"], True),
    (["King", "Knight"], False)
]

for test_path, right_answer in is_path_tests:
    print(test_path)
    print("FUNCTION ANSWER:", graph.is_path_by_names(test_path))
    print("RIGHT ANSWER:", right_answer)

print('is full subgraph', graph.is_full_subgraph_by_names(["King", "Queen"]))
print('is full subgraph', graph.is_full_subgraph_by_names(["Pawn", "Rook", "Knight"]))
print('is full subgraph', graph.is_full_subgraph_by_names(["King"]))
print('is full subgraph', graph.is_full_subgraph_by_names(["Pawn", "Rook", "Knight", "Bishop"]))

[0, 3]
[2]
[0, 2, 3]
Count of edges: 10
1
[0, 1, 2, 3, 4, 5]
[3]
[2, 3, 4, 5]
[0, 2, 3, 4, 5]
['King', 'Queen', 'Bishop', 'Knight', 'Rook', 'Pawn']
FUNCTION ANSWER: True
RIGHT ANSWER: True
['King', 'Pawn']
FUNCTION ANSWER: True
RIGHT ANSWER: True
['King']
FUNCTION ANSWER: True
RIGHT ANSWER: True
['King', 'Knight']
FUNCTION ANSWER: False
RIGHT ANSWER: False
is full subgraph True
is full subgraph True
is full subgraph True
is full subgraph False


## Матрица смежности

In [ ]:
class AdjacencyMatrixGraph:
    def __init__(self, adjacency_matrix: list[list[int]], labels: list[str]):
        self.adjacency_matrix = adjacency_matrix
        self.labels = labels

    def get_neighbours_by_index(self, index: int) -> list[int]:
        kids = self.get_kids_by_index(index)
        parents = self.get_parents_by_index(index)
        return sorted(kids + parents)

    def get_kids_by_index(self, index: int) -> list[int]:
        kids = []
        for line in range(len(self.labels)):
            if self.adjacency_matrix[index][line] > 0:
                kids.append(line)
        return kids

    def get_parents_by_index(self, index: int) -> list[int]:
        parents = []
        for line in range(len(self.labels)):
            if self.adjacency_matrix[line][index] > 0:
                parents.append(line)
        return parents

    def is_path_by_names(self, path: list[str]) -> bool:
        path = [self.get_index_by_name(name) for name in path]
        return self.is_path_by_indexes(path)

    def is_path_by_indexes(self, path: list[int]) -> bool:
        edges = set()
        for index in range(1, len(path)):
            parent, kid = path[index-1], path[index]
            if self.adjacency_matrix[parent][kid] == 0:
                return False
            edge = (parent, kid)
            if edge in edges:
                return False
            edges.add(edge)
        return True

    def get_index_by_name(self, name: str) -> int:
        for index, value in enumerate(self.labels):
            if value == name:
                return index
        raise ValueError

    def get_indexes_sum_incident_edges_more_than(self, criterion: int) -> list[int]:
        filtered_vertexes = []
        for vertex in range(len(self.adjacency_matrix)):
            sum_incident_edges = 0
            for line in range(len(self.adjacency_matrix)):
                sum_incident_edges += self.adjacency_matrix[line][vertex]
                sum_incident_edges += self.adjacency_matrix[vertex][line]
            if sum_incident_edges > criterion:
                filtered_vertexes.append(vertex)
        return filtered_vertexes

    def get_count_of_edges(self) -> int:
        count = 0
        for i in range(len(self.adjacency_matrix)):
            for j in range(len(self.adjacency_matrix)):
                if self.adjacency_matrix[i][j] > 0:
                    count += 1
        return count

In [5]:
graph = AdjacencyMatrixGraph(adjacency_matrix, labels)
print(graph.get_kids_by_index(1))          # [0, 3]
print(graph.get_parents_by_index(1))       # [2]
print(graph.get_neighbours_by_index(1))    # [0, 2, 3]

print("Count of edges:", graph.get_count_of_edges())  # 10

print(graph.get_index_by_name("Knight"))   # 1

print(graph.get_indexes_sum_incident_edges_more_than(0))  # [0, 5]
print(graph.get_indexes_sum_incident_edges_more_than(12))  # 3
print(graph.get_indexes_sum_incident_edges_more_than(7))  # 2, 3, 4, 5
print(graph.get_indexes_sum_incident_edges_more_than(6))  # 0, 2, 3, 4, 5

is_path_tests = [
    (["King", "Queen", "Bishop", "Knight", "Rook", "Pawn"], True),
    (["King", "Pawn"], True),
    (["King"], True),
    (["King", "Knight"], False)
]

for test_path, right_answer in is_path_tests:
    print(test_path)
    print("FUNCTION ANSWER:", graph.is_path_by_names(test_path))
    print("RIGHT ANSWER:", right_answer)

NameError: name 'AdjacencyMatrixGraph' is not defined

## Список ребер

In [ ]:
from collections import namedtuple, defaultdict


Edge = namedtuple("Edge", ["parent", "kid", "weight"])


class ListEdgesGraph:
    def __init__(self, adjacency_matrix: list[list[int]], labels: list[str]):
        self.labels = labels
        self.edges: list[Edge] = []
        self.set_of_edges = set()
        self.__init_list_of_edges(adjacency_matrix)

    def __init_list_of_edges(self, adjacency_matrix) -> None:
        for parent_id in range(len(self.labels)):
            for kid_id in range(len(self.labels)):
                edge = Edge(parent_id, kid_id, adjacency_matrix[parent_id][kid_id])
                if edge.weight:
                    self.edges.append(edge)
                    self.set_of_edges.add((parent_id, kid_id))

    def get_neighbours_by_index(self, index: int) -> list[int]:
        kids = self.get_kids_by_index(index)
        parents = self.get_parents_by_index(index)
        return sorted(kids + parents)

    def get_kids_by_index(self, parent_index: int) -> list[int]:
        kids = []
        for edge in self.edges:
            if edge.parent == parent_index:
                kids.append(edge.kid)
        return kids

    def get_parents_by_index(self, kid_index: int) -> list[int]:
        parents = []
        for edge in self.edges:
            if edge.kid == kid_index:
                parents.append(edge.parent)
        return parents

    def is_path_by_names(self, path: list[str]) -> bool:
        path = [self.get_index_by_name(name) for name in path]
        return self.is_path_by_indexes(path)

    def is_path_by_indexes(self, path: list[int]) -> bool:
        path_edges = set()
        for index in range(1, len(path)):
            edge = (path[index-1], path[index])
            if edge in path_edges:
                return False
            path_edges.add((path[index-1], path[index]))
        return path_edges.issubset(self.set_of_edges)

    def get_index_by_name(self, name: str) -> int:
        for index, value in enumerate(self.labels):
            if value == name:
                return index
        raise ValueError

    def get_indexes_sum_incident_edges_more_than(self, criterion: int) -> list[int]:
        vertexes_sum_incident_edges = defaultdict(int)
        for edge in self.edges:
            vertexes_sum_incident_edges[edge.parent] += edge.weight
            vertexes_sum_incident_edges[edge.kid] += edge.weight
        filtered_vertexes = []
        for vertex, sum_incident_edges in vertexes_sum_incident_edges.items():
            if sum_incident_edges > criterion:
                filtered_vertexes.append(vertex)
        return sorted(filtered_vertexes)

    def get_count_of_edges(self) -> int:
        return len(self.edges)

In [ ]:
graph = ListEdgesGraph(adjacency_matrix, labels)

print(graph.edges)
print(graph.set_of_edges)

print(graph.get_kids_by_index(1))          # [0, 3]
print(graph.get_parents_by_index(1))       # [2]
print(graph.get_neighbours_by_index(1))    # [0, 2, 3]

print("Count of edges:", graph.get_count_of_edges())  # 10

print(graph.get_index_by_name("Knight"))   # 1

print(graph.get_indexes_sum_incident_edges_more_than(0))  # 0, 1, 2, 3, 4, 5
print(graph.get_indexes_sum_incident_edges_more_than(12))  # 3
print(graph.get_indexes_sum_incident_edges_more_than(7))  # 2, 3, 4, 5
print(graph.get_indexes_sum_incident_edges_more_than(6))  # 0, 2, 3, 4, 5

is_path_tests = [
    (["King", "Queen", "Bishop", "Knight", "Rook", "Pawn"], True),
    (["King", "Pawn"], True),
    (["King"], True),
    (["King", "Knight"], False)
]

for test_path, right_answer in is_path_tests:
    print(test_path)
    print("FUNCTION ANSWER:", graph.is_path_by_names(test_path))
    print("RIGHT ANSWER:", right_answer)

## Sizes of objects

In [ ]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

print("list of edges:", get_size(ListEdgesGraph(adjacency_matrix, labels)))
print("adjacency matrix:", get_size(AdjacencyMatrixGraph(adjacency_matrix, labels)))
print("array of recors:", get_size(ArrayRecordsGraph(adjacency_matrix, labels)))
print("integer:", sys.getsizeof(12))

## Замеры времени работы

In [ ]:
from datetime import datetime

def test_get_neighbours(name, obj):
    global COUNT_OF_OPERATIONS
    vertex_count = len(obj.labels)
    start_time = datetime.now()
    for test in range(COUNT_OF_OPERATIONS):
        obj.get_neighbours_by_index(test % vertex_count)
    end_time = datetime.now()
    print("get_neighbours:", end_time - start_time)

def test_get_count_of_edges(name, obj):
    global COUNT_OF_OPERATIONS
    start_time = datetime.now()
    for test in range(COUNT_OF_OPERATIONS):
        obj.get_count_of_edges()
    end_time = datetime.now()
    print("get_count_of_vertex:", end_time - start_time)

def test_get_indexes_sum_incident_edges_more_than(name, obj):
    global COUNT_OF_OPERATIONS, MAX_CRITERION
    start_time = datetime.now()
    for test in range(COUNT_OF_OPERATIONS):
        obj.get_indexes_sum_incident_edges_more_than(test % MAX_CRITERION)
    end_time = datetime.now()
    print("get_indexes_sum_incident_edges_more_than:", end_time - start_time)

def test_is_path(name, obj):
    is_path_tests = [
        (["King", "Queen", "Bishop", "Knight", "Rook", "Pawn"], True),
        (["King", "Pawn"], True),
        (["King"], True),
        (["King", "Knight"], False)
    ]
    start_time = datetime.now()
    for test in range(COUNT_OF_OPERATIONS):
        test_id = test % len(is_path_tests)
        obj.is_path_by_names(is_path_tests[test_id][0])
    end_time = datetime.now()
    print("is path:", end_time - start_time)

COUNT_OF_OPERATIONS = 10 ** 6
MAX_CRITERION = 20

name_to_object = {
    "list of edges:": ListEdgesGraph(adjacency_matrix, labels),
    "adjacency matrix:": AdjacencyMatrixGraph(adjacency_matrix, labels),
    "array of recors:": ArrayRecordsGraph(adjacency_matrix, labels)
}

for name, obj in name_to_object.items():
    print("test", name)
    test_get_neighbours(name, obj)
    test_get_count_of_edges(name, obj)
    test_get_indexes_sum_incident_edges_more_than(name, obj)
    test_is_path(name, obj)
    print('---')
print('end')